In [2]:
import json
from dependencies.handle_gre_data import GREDataProcessor

# Define configuration
config = {
    'traders': ['Beimnet', 'Bouchra', 'Eduardo', 'Felman'],
    'strategies': {
        'BTC': ['Core BTC', 'Options $', 'RelativeValue BTC'],
        'ETH': ['Core ETH', 'Options $', 'RelativeValue ETH'],
        'SOL': ['Core SOL', 'Options $', 'RelativeValue SOL'],
        'Alts': ['Core', 'Options', 'RelativeValue']
    }
}

# Create a DeltaProcessor instance
processor = GREDataProcessor(config)

# Process the CSV file
csv_filepath = '../data/GDLP GRE Data 29072024 (Midday).csv'
final_df = processor.process_csv(csv_filepath)

# Save the results
final_df.to_csv('../output/29072024(Midday).csv', index=False)
final_df.to_excel('../output/29072024(Midday).xlsx', index=False)
print(final_df)


AttributeError: 'GREDataProcessor' object has no attribute 'process_csv'

In [3]:
from dependencies.construct_frontpage_time_series import ConstructFrontpageTimeSeries

base_directory = "/mnt/box/Risk Report/2024"
novo_filename_pattern = "Frontpage - (Novo) - 15072024"
gdlp_filename_pattern = "Frontpage - Macro Report - No Crypto Options - 17072024"

ticker_groups = {'BTC': ['BTC']}

metrics = ['Mkt Val (ex DLOM) T-0 ($)']

analysis = ConstructFrontpageTimeSeries(base_directory)
novo_frontpage_time_series = analysis.collect_data(novo_filename_pattern, ticker_groups, metrics, preprocess_tickers=True)
novo_frontpage_time_series.to_csv('../output/novo_frontpages_time_series.csv')

In [1]:
from dependencies.combine_frontpage_gre import CombineFrontpageGRE

# Define configuration for HandleGREData
config = {
    'traders': ['Beimnet', 'Bouchra', 'Eduardo', 'Felman'],
    'strategies': {
        'BTC': ['Core BTC', 'Options $', 'RelativeValue BTC'],
        'ETH': ['Core ETH', 'Options $', 'RelativeValue ETH'],
        'SOL': ['Core SOL', 'Options $', 'RelativeValue SOL'],
        'Alts': ['Core', 'Options', 'RelativeValue']
    }
}

base_directory = "/mnt/box/Risk Report/2024"
gre_data_filepath = '../data/GDLP GRE Data 29072024.csv'

# Initialize CombineFrontpageGRE
combine_processor = CombineFrontpageGRE(base_directory, gre_data_filepath)

# Run the combined process
combine_processor.run()


Extracted date string: 26072024
522033.7424640002
-997379.111775978
-473518.8306739699
538834.3686203668
234583.187
-1438778.1345525
-422935.60317000025
-375163.629984
848095.8490655487
524072.9207345735
329280.155671
-720842.6300000004
2011389.0388127551
-477735.4211573556
1172543.650248211
-638597.7998376
-3104942.410710016
998655.59826
2559358.073622
-2051725.59
-397565.521314
        Beimnet BTC Options Delta  Beimnet BTC Delta  Beimnet ETH Core Delta  \
Values               8.210123e+06       8.210123e+06           -1.302728e+07   

        Beimnet ETH Delta  Beimnet SOL Core Delta  Beimnet SOL Delta  \
Values      -1.302728e+07          -718328.080115     -718328.080115   

        Beimnet Alts Core Delta  Beimnet Alts Delta  Beimnet Delta  \
Values            -2.412324e+06       -2.412324e+06  -7.947804e+06   

        Bouchra BTC Options Delta  ...  GDLP BTC Total Gamma  \
Values               5.430703e+06  ...          5.862142e+06   

        GDLP ETH Total Gamma  GDLP SOL To

In [4]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Load the CSV data into a DataFrame
csv_filepath = '../output/23072024_with_novo.csv'
df = pd.read_csv(csv_filepath)

# Load the Excel template
template_path = '../data/Template.xlsx'
wb = load_workbook(template_path)

# Print the sheet names to find the correct one
print(wb.sheetnames)

# Replace 'CorrectSheetName' with the actual sheet name from the template
template_sheet_name = 'Sheet1'
template_sheet = wb[template_sheet_name]

# Get the sheet where the CSV data will be placed
new_sheet_name = 'CSV Data'
if new_sheet_name in wb.sheetnames:
    data_sheet = wb[new_sheet_name]
else:
    data_sheet = wb.create_sheet(title=new_sheet_name)

# Write the DataFrame to the new sheet
for r in dataframe_to_rows(df, index=False, header=True):
    data_sheet.append(r)

# Define a mapping from CSV columns to template cells for Delta values
delta_mapping = {
    'Novo BTC Delta': 'C4',
    'GDLP ex-Novo BTC Delta': 'C5',
    'Novo ETH Delta': 'C8',
    'GDLP ex-Novo ETH Delta': 'C9',
    'Novo SOL Delta': 'C12',
    'GDLP ex-Novo SOL Delta': 'C13',
    'Novo Alts Delta': 'C16',
    'GDLP ex-Novo Alts Delta': 'C17',
    'Passive Beta Delta': 'C19'
}

# Define a mapping from CSV columns to template cells for Gamma values
gamma_mapping = {
    'Novo BTC Options Gamma': 'F4',
    'GDLP ex-Novo BTC Gamma': 'F5',
    'Novo ETH Options Gamma': 'F8',
    'GDLP ex-Novo ETH Gamma': 'F9',
    'Novo SOL Options Gamma': 'F12',
    'GDLP ex-Novo SOL Gamma': 'F13',
    'Novo Alts Gamma': 'F16',
    'GDLP ex-Novo Alts Gamma': 'F17',
}

# Define a mapping from CSV columns to template cells for Vega values
vega_mapping = {
    'Novo BTC Options Vega': 'G4',
    'GDLP ex-Novo BTC Vega': 'G5',
    'Novo ETH Options Vega': 'G8',
    'GDLP ex-Novo ETH Vega': 'G9',
    'Novo SOL Options Vega': 'G12',
    'GDLP ex-Novo SOL Vega': 'G13',
    'Novo Alts Vega': 'G16',
    'GDLP ex-Novo Alts Vega': 'G17',
}

# Fill the template cells with corresponding Delta data from the CSV
for column, cell in delta_mapping.items():
    if column in df.columns:
        value = df[column].values[0]
        template_sheet[cell] = value

# Fill the template cells with corresponding Gamma data from the CSV
for column, cell in gamma_mapping.items():
    if column in df.columns:
        value = df[column].values[0]
        template_sheet[cell] = value

# Fill the template cells with corresponding Vega data from the CSV
for column, cell in vega_mapping.items():
    if column in df.columns:
        value = df[column].values[0]
        template_sheet[cell] = value

# # Also include {Trader} {Asset} Delta, Gamma, and Vega
# trader_asset_mappings = {
#     'Delta': delta_mapping,
#     'Gamma': gamma_mapping,
#     'Vega': vega_mapping
# }

# for trader in ['Beimnet', 'Bouchra', 'Eduardo', 'Felman']:
#     for asset in ['BTC', 'ETH', 'SOL']:
#         for greek, mapping in trader_asset_mappings.items():
#             column_name = f'{trader} {asset} {greek}'
#             if column_name in df.columns:
#                 cell = f"{chr(67 + len(mapping))}{4 + len(mapping)}"  # Adjust column index and row
#                 template_sheet[cell] = df[column_name].values[0]
#                 mapping[column_name] = cell

# Save the workbook with the filled-in data
output_path = "../output/29072024.xlsx"
wb.save(output_path)


['Sheet1']
